In [2]:
import sys

sys.tracebacklimit = None

# Parameters & constants

In [1]:
from pathlib import Path

lx = 5  # length in x direction (Å)
ly = 5  # length in y direction (Å)
lz = 10 # length in z direction (Å)
nn = 1  # number of nearest neighbors | don't use 0!

basepath = Path(f"output/example/len_{lx}x{ly}/nn_{nn}")
basepath.mkdir(parents=True, exist_ok=True)

# Define structure

In [2]:
from tb_hamiltonian.utils import get_structure

structure = get_structure(
    unit_cell_filepath=Path("output/example/POSCAR"),  # local unit cell file
    lengths=(lx, ly, lz),
)

structure.info["label"] = "BLG"  # will show up at top of Hamiltonian output file

structure.write(basepath / "POSCAR", format="vasp")

# Compute H

In [3]:
from tb_hamiltonian import TightBindingHamiltonian

H = TightBindingHamiltonian(
    structure=structure,
    nearest_neighbor=nn,
    distances=(0.0, 1.425, 2.468, 2.850),
    hopping_parameters=(0.0, -2.7, 0.083, -0.27),
    interlayer_coupling=0.33,
)

H.build()

# Apply onsite term

In [4]:
from tb_hamiltonian.potentials import PotentialFactory

potential = PotentialFactory("null")
potential.amplitude = 1.0
potential.width = 0.0

H.update_onsite_terms(
    onsite_term=0.0,
    potential=potential,
    alpha=(1.0, 0.5),
)

path = basepath / f"{potential.name}_a_{potential.amplitude}_w_{potential.width}"
path.mkdir(parents=True, exist_ok=True)

# Sanity checks


In [70]:
# H.onsite_count()

In [71]:
# H.interaction_counts()

# Write H to file

In [72]:
H.write_to_file(path)

# Plotting

In [73]:
# H.plot_matrix()

In [74]:
# H.plot_grid()